# OPTaaS Quick Start

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

## Connect to OPTaaS using your API Key

In [1]:
library(optaas.client)

client <- OPTaaSClient$new("https://optaas.mindfoundry.ai", "Your OPTaaS API Key")

## Define your parameters

In [2]:
parameters <- list(
    BoolParameter('my_bool'),
    CategoricalParameter('my_cat', values=list('a', 'b', 'c'), default='c'),
    ChoiceParameter('ints_or_floats', choices=list(
        GroupParameter('ints', items=list(
            IntParameter('my_int', minimum=0, maximum=20),
            IntParameter('my_optional_int', minimum=-10, maximum=10, optional=TRUE)
        )),
        GroupParameter('floats', items=list(
            FloatParameter('float1', minimum=0, maximum=1),
            FloatParameter('float2', minimum=0.5, maximum=4.5)
        ))
    ))
)

## Define your scoring function

The argument names in your scoring function must match the parameter names you defined above.

In [3]:
scoring_function <- function(my_bool, my_cat, ints_or_floats) {
    score <- if (isTRUE(my_bool)) 5 else -5
    score <- if (my_cat == 'a') score + 1 else score + 3
    if (!is.null(ints_or_floats$ints)) {
        score <- score + do.call(sum, ints_or_floats$ints)
    } else {
        score <- score * do.call(sum, ints_or_floats$floats)
    }
    score
}

## Create your Task

In [4]:
task <- client$create_task(
    title="Dummy task",
    parameters=parameters,
    goal="min",  # optional (default is "max")
    min_known_score=-22, max_known_score=44  # optional
)

## Run your Task

In [5]:
best_result <- task$run(scoring_function=scoring_function, number_of_iterations=20)

print(paste("Best Score:", best_result$score))
print("with configuration:")
print(best_result$configuration$values)

[1] "Iteration: 1   Score: 8"
[1] "Iteration: 2   Score: -6"
[1] "Iteration: 3   Score: 8"
[1] "Iteration: 4   Score: 11"
[1] "Iteration: 5   Score: 14"
[1] "Iteration: 6   Score: -5.67974962470235"
[1] "Iteration: 7   Score: -9"
[1] "Iteration: 8   Score: -10.2342840194121"
[1] "Iteration: 9   Score: 17.9309452036351"
[1] "Iteration: 10   Score: 28"
[1] "Iteration: 11   Score: -3.71144029257866"
[1] "Iteration: 12   Score: -6.79407605413269"
[1] "Iteration: 13   Score: 8"
[1] "Iteration: 14   Score: 13.9716469365809"
[1] "Iteration: 15   Score: -4.85055097677501"
[1] "Iteration: 16   Score: -9.44777782922238"
[1] "Iteration: 17   Score: -5.8358581073926"
[1] "Iteration: 18   Score: -2.65079383869822"
[1] "Iteration: 19   Score: -11"
[1] "Iteration: 20   Score: -10"
[1] "Best Score: -11"
[1] "with configuration:"
$my_bool
[1] FALSE

$my_cat
[1] "a"

$ints_or_floats
$ints_or_floats$ints
$ints_or_floats$ints$my_int
[1] 0

$ints_or_floats$ints$my_optional_int
[1] -7



